In [37]:
#%pip install pandas
%pip install mlflow

  Using cached mlflow-3.5.1-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-3.5.1-py3-none-any.whl.metadata (31 kB)
  Using cached mlflow_tracing-3.5.1-py3-none-any.whl.metadata (19 kB)
  Using cached flask_cors-6.0.1-py3-none-any.whl.metadata (5.3 kB)
  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached alembic-1.17.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached cryptography-46.0.3-cp311-abi3-macosx_10_9_universal2.whl.metadata (5.7 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached gunicorn-23.0.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached pyarrow-21.0.0-cp313-cp313-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached scikit_learn-1.7.2-cp313-cp313-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached sqlalchemy-2.0.44-cp313-cp313-macosx_11_0_arm64.whl.met

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim 
from torchvision import datasets, transforms
import pandas as pd


transform = transforms.Compose([
    transforms.ToTensor(),
])


train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [14]:
# Model

class NN(nn.Module): # class 

    def __init__(self): # Constructor
        super(NN, self).__init__() # initialization of the superclass
        self.layer1 = nn.Linear(28*28, 64) # input layer
        self.layer2 = nn.Linear(64, 32) # hidden layer
        self.layer3 = nn.Linear(32, 10)  # output layer
    
    def forward(self,x): # propagate the information through the network 
        x = x.view(-1, 28*28) # flatten (2D -> 1D)
        x = torch.relu(self.layer1(x)) # activation using ReLU 
        x = torch.relu(self.layer2(x)) 
        x = self.layer3(x) # identity activation -> logit
        return x


In [31]:
model = NN()
lr = 1e-4 # lerning rate
loss = nn.CrossEntropyLoss() # CE because multi-class problem 
# optimizer = optim.SGD(model.parameters(), lr = lr) # stochastic gradient descent
optimizer = optim.Adam(model.parameters(), lr = lr) # modern version of stochastic gradient descent
n_epochs = 20

for epoch in range(n_epochs): # training loop
    model.train() # train mode 
    running_loss = 0.0 # loss per epoch 
    for images,labels in train_loader:
        optimizer.zero_grad() # reset gradient 
        # forward 
        outputs = model(images) # calculate outputs
        curr_loss = loss(outputs, labels) # loss between output and label 
        running_loss += curr_loss
        # backward 
        curr_loss.backward() # gradients
        optimizer.step() # Update the weights 
    print(f"Epoch [{epoch +1}/{n_epochs}], Loss: {running_loss}")
    

Epoch [1/20], Loss: 1070.5306396484375
Epoch [2/20], Loss: 581.562255859375
Epoch [3/20], Loss: 504.1989440917969
Epoch [4/20], Loss: 465.3745422363281
Epoch [5/20], Loss: 441.0765686035156
Epoch [6/20], Loss: 422.99078369140625
Epoch [7/20], Loss: 410.5953674316406
Epoch [8/20], Loss: 400.0853576660156
Epoch [9/20], Loss: 391.016845703125
Epoch [10/20], Loss: 384.3331298828125
Epoch [11/20], Loss: 377.7515563964844
Epoch [12/20], Loss: 372.02764892578125
Epoch [13/20], Loss: 366.339111328125
Epoch [14/20], Loss: 361.4666442871094
Epoch [15/20], Loss: 356.7622375488281
Epoch [16/20], Loss: 353.1795654296875
Epoch [17/20], Loss: 348.7491455078125
Epoch [18/20], Loss: 345.4549865722656
Epoch [19/20], Loss: 341.9302978515625
Epoch [20/20], Loss: 338.5642395019531


In [32]:
# Evaluation 

model.eval() # setting the model to evaluation mode (implementation optimization)
correct = 0
total = 0

with torch.no_grad(): # we are not interested in gradient anymore 
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[-1] 
        total += labels.size(0) 
        correct += (predicted==labels).sum().item()

accuracy = correct/total
print(f"Accuracy: {accuracy*100:.2f}%")



Accuracy: 86.03%


In [35]:
len(train_dataset)

60000

In [36]:
len(test_dataset)

10000

In [33]:
torch.set_printoptions(precision=2,sci_mode=False)
print(f"{nn.functional.softmax(outputs, dim = 1)}")

tensor([[0.05, 0.00, 0.00, 0.93, 0.00, 0.00, 0.01, 0.00, 0.01, 0.00],
        [0.08, 0.46, 0.09, 0.17, 0.07, 0.00, 0.13, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.99, 0.00, 0.01],
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
        [0.00, 0.00, 0.52, 0.00, 0.41, 0.00, 0.07, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00, 0.00, 0.00, 0.00],
        [0.14, 0.00, 0.36, 0.06, 0.02, 0.00, 0.42, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
        [0.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00],
        [0.00, 0.99, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
        [0.09, 0.00, 0.00, 0.05, 0.01, 0.00, 0.06, 0.00, 0.79, 0.00],
        [0.00, 1.00,

In [34]:
labels

tensor([3, 2, 7, 5, 8, 4, 5, 6, 8, 9, 1, 9, 1, 8, 1, 5])